In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import scipy.ndimage as nd
from tensorflow import keras
from keras import backend as K
import pdb
from tensorflow.keras import layers

%matplotlib inline
import pylab as pl
from IPython import display
from sklearn.metrics import accuracy_score
import tensorflow_datasets as tfds

In [ ]:
tf.random.set_seed(1)

In [ ]:
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = tf.keras.datasets.mnist.load_data()

In [ ]:
print('MNIST Train Data Shape:', mnist_train_images.shape)
print('MNIST Test Data Shape:', mnist_test_images.shape)
print('MNIST Train Label Shape:', mnist_train_labels.shape)
print('MNIST Test Label Shape:', mnist_test_labels.shape)

In [ ]:
from scipy.io import loadmat as load
svhn_train = load('/kaggle/input/svhn-mat/train_32x32.mat')
svhn_test = load('/kaggle/input/svhn-mat/test_32x32.mat')
print('SVHN Train Data Shape:', svhn_train['X'].shape)
print('SVHN Test Data Shape:', svhn_test['X'].shape)
print('SVHN Train Label Shape:', svhn_train['y'].shape)
print('SVHN Test Label Shape:', svhn_test['y'].shape)

In [ ]:
svhn_train_images = svhn_train['X']
svhn_train_labels = svhn_train['y']
svhn_test_images = svhn_test['X']
svhn_test_labels = svhn_test['y']

In [ ]:
##svhn labels: 1-10 -> 0-9
svhn_train_labels = svhn_train_labels%10
svhn_test_labels = svhn_test_labels%10
##svhn images (图片高，图片宽，通道数，图片数)->(图片数,图片高，图片宽，通道数)
svhn_train_images = np.transpose(svhn_train_images, (3, 0, 1, 2))
svhn_test_images = np.transpose(svhn_test_images, (3, 0, 1, 2))

In [ ]:
print('SVHN Train Data Shape:', svhn_train_images.shape)
print('SVHN Test Data Shape:', svhn_test_images.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(svhn_train_images[1])

In [ ]:
import torch
idx1 = torch.load('/kaggle/input/train-test-idx/train-ms-mnist-idx.pt')
idx2 = torch.load('/kaggle/input/train-test-idx/train-ms-svhn-idx.pt')
test_idx1 = torch.load('/kaggle/input/train-test-idx/test-ms-mnist-idx.pt')
test_idx2 = torch.load('/kaggle/input/train-test-idx/test-ms-svhn-idx.pt')
ofd_test_idx1 = torch.load('/kaggle/input/train-test-idx/ofd-test-ms-emnist-idx.pt')
ofd_test_idx2 = torch.load('/kaggle/input/train-test-idx/ofd-test-ms-chars74k-idx.pt')

In [ ]:
modal_mnist_samples = mnist_train_images[idx1]
modal_svhn_samples = svhn_train_images[idx2]
modals_labels = mnist_train_labels[idx1]
print(modals_labels.shape)
print(len(test_idx1))

In [ ]:
modal_mnist_tests = mnist_test_images[test_idx1]
modal_svhn_tests = svhn_test_images[test_idx2]
modals_test_labels = mnist_test_labels[test_idx1]

In [ ]:
plt.imshow(modal_mnist_tests[1])
plt.imshow(modal_svhn_tests[1])

In [ ]:
def dense_to_one_hot(labels_dense, num_classes=10):
    """Convert class labels from scalars to one-hot vectors"""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot
mnist_train_labels_hot = dense_to_one_hot(mnist_train_labels)
mnist_test_labels_hot = dense_to_one_hot(mnist_test_labels)
svhn_train_labels_hot = dense_to_one_hot(svhn_train_labels)
svhn_test_labels_hot = dense_to_one_hot(svhn_test_labels)
modals_labels_hot = dense_to_one_hot(modals_labels)
modals_test_labels_hot = dense_to_one_hot(modals_labels)

In [ ]:
k=10
def KL(alpha):
    beta=tf.constant(np.ones((1,k)),dtype=tf.float32)
    S_alpha = tf.reduce_sum(alpha,axis=1,keepdims=True)
    S_beta = tf.reduce_sum(beta,axis=1,keepdims=True)
    lnB = tf.math.lgamma(S_alpha) - tf.reduce_sum(tf.math.lgamma(alpha),axis=1,keepdims=True)
    lnB_uni = tf.reduce_sum(tf.math.lgamma(beta),axis=1,keepdims=True) - tf.math.lgamma(S_beta)
    
    dg0 = tf.math.digamma(S_alpha)
    dg1 = tf.math.digamma(alpha)
    
    kl = tf.reduce_sum((alpha - beta)*(dg1-dg0),axis=1,keepdims=True) + lnB + lnB_uni
    print(kl)
    return kl
class CustomMSE(keras.losses.Loss):
    def __init__(self, annealing_coef=tf.Variable(initial_value=0.0,name='annealing_coef', trainable=False), name="custom_mse"):
      super().__init__(name=name)
      self.annealing_coef = annealing_coef
      self.A = 0
      self.B = 0
      self.C = 0
      #print('Inital:', self.annealing_coef)
    def call(self, y_true, y_pred):
      y_pred = tf.cast(y_pred, tf.float32)
      y_true = tf.cast(y_true, tf.float32)
      S = tf.reduce_sum(y_pred, axis=1, keepdims=True) 
      E = y_pred - 1
      m = y_pred / S
      #print('y_true:',y_true)
      A = tf.reduce_sum((y_true-m)**2, axis=1, keepdims=True) 
      B = tf.reduce_sum(y_pred*(S-y_pred)/(S*S*(S+1)), axis=1, keepdims=True)       
      alp = E*(1-y_true) + 1 
      #print('Step:',self.annealing_coef)
      C =  self.annealing_coef * KL(alp)
      self.A = A
      self.B = B
      self.C = C
      print(A,B,C)
      return tf.reduce_mean((A + B) + C)
class WarmUp(keras.callbacks.Callback):
    """
    继承Callback，实现对annealing_rate的调度
    """
    def __init__(self,total_steps,global_step_init=0,ac=0,
                 verbose=1):
        super(WarmUp, self).__init__()
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.ac = ac
        self.verbose = verbose
        #learning_rates用于记录每次更新后的学习率，方便图形化观察
        self.acs = []
	#更新global_step，并记录当前退火系数
    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        ac = K.get_value(self.model.loss.annealing_coef)
        self.acs.append(ac)
	#更新退火系数
    def on_batch_begin(self, batch, logs=None):
        ac = tf.minimum(1.0,tf.cast(self.global_step/self.total_steps,tf.float32)).numpy()
        #print('ac:',ac)
        K.set_value(self.model.loss.annealing_coef, ac)
        #print('after set',self.model.loss.annealing_coef)
        if self.verbose > 0:
          print('\nBatch %05d: setting annealing_coef '
                  'rate to %s.' % (self.global_step + 1, ac))


In [ ]:
def conv2d_bn(inpt, filters=64, kernel_size=(3,3), strides=1, padding='same'):
    '''卷积、归一化和relu三合一'''
    x = layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(inpt)
    x = layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def basic_bottle(inpt, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False):
    '''18中的4个basic_bottle'''
    x = conv2d_bn(inpt, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)
    x = conv2d_bn(x, filters=filters)
    if if_baisc==True:
        temp = conv2d_bn(inpt, filters=filters, kernel_size=(1,1), strides=2, padding='same')
        outt = layers.add([x, temp])
    else:
        outt = layers.add([x, inpt])
    return outt

def resnet18(class_nums,input_shape=(28,28,1)):
    '''主模型'''
    inpt = layers.Input(shape=input_shape)
    #layer 1
    x = conv2d_bn(inpt, filters=64, kernel_size=(7,7), strides=2, padding='valid')
    x = layers.MaxPool2D(pool_size=(3,3), strides=2)(x)
    #layer 2
    x = basic_bottle(x, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    x = basic_bottle(x, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    #layer 3
    x = basic_bottle(x, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 4
    x = basic_bottle(x, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 5
    x = basic_bottle(x, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    #GlobalAveragePool
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(class_nums, activation='softmax')(x)
    model = tf.keras.Model(inputs=inpt, outputs=x)
    return model

In [ ]:
model_mnist = resnet18(class_nums=10,input_shape=(28,28,1))
#model_mnist.summary()
model_mnist.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model_mnist.fit(modal_mnist_samples, modals_labels, batch_size=64, epochs=10) 
mnist_test_preds = model_mnist.predict(modal_mnist_tests)
y_mnist_preds = np.argmax(mnist_test_preds, axis=1)
acc_score_mnist = accuracy_score(y_true=modals_test_labels,y_pred=y_mnist_preds)
print(acc_score_mnist)

In [ ]:
model_svhn = resnet18(class_nums=10,input_shape=(32,32,3))
#model_svhn.summary()
model_svhn.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model_svhn.fit(modal_svhn_samples, modals_labels, batch_size=64, epochs=10) 
svhn_test_preds = model_svhn.predict(modal_svhn_tests)
y_svhn_preds = np.argmax(svhn_test_preds, axis=1)
acc_score_svhn = accuracy_score(y_true=modals_test_labels,y_pred=y_svhn_preds)
print(acc_score_svhn)

### Out of distribution testing

In [ ]:
(emnist_images, emnist_labels) =  tfds.as_numpy(tfds.load('emnist/letters',
                         split = 'test', 
                         batch_size=-1, 
                         as_supervised=True))
chars74k_images = np.load('/kaggle/input/chars74k/EnglishImages.npy')
chars74k_labels = np.load('/kaggle/input/chars74k/EnglishLabels.npy')
ofd_emnist_tests = emnist_images[ofd_test_idx1]
ofd_chars74k_tests = chars74k_images[ofd_test_idx2]
ofd_test_labels = emnist_labels[ofd_test_idx1]

In [ ]:
mnist_ofd_test_preds = model_mnist.predict(ofd_emnist_tests)

In [ ]:
def resnet18EDL1(class_nums,input_shape=(28,28,1)):
    '''主模型'''
    inpt = layers.Input(shape=input_shape)
    #layer 1
    x = conv2d_bn(inpt, filters=64, kernel_size=(7,7), strides=2, padding='valid')
    x = layers.MaxPool2D(pool_size=(3,3), strides=2)(x)
    #layer 2
    x = basic_bottle(x, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    x = basic_bottle(x, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    #layer 3
    x = basic_bottle(x, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 4
    x = basic_bottle(x, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 5
    x = basic_bottle(x, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    #GlobalAveragePool
    x = layers.GlobalAveragePooling2D()(x)
    e = layers.Dense(class_nums, activation='relu')(x)
    b = layers.Dense(class_nums, activation='softmax')(x)
    E = tf.reduce_sum(e, axis=1, keepdims=True)
    alpha = E * b+1
    model = tf.keras.Model(inputs=inpt, outputs=alpha)
    return model

In [ ]:
model_mnist_edl1 = resnet18EDL1(class_nums=10,input_shape=(28,28,1))
#model_mnist.summary()
annealing_step = 10*(len(modals_labels)//64)
model_mnist_edl1.compile(optimizer=tf.keras.optimizers.Adam(),loss=CustomMSE(), metrics=['accuracy'])
warm_up_ac = WarmUp(total_steps=annealing_step,global_step_init=0,ac=0,verbose=0)
history_edl_mnist1=model_mnist_edl1.fit(modal_mnist_samples, modals_labels_hot, batch_size=64, epochs=10,verbose=1,callbacks=[warm_up_ac])
mnist_test_preds1 = model_mnist_edl1.predict(modal_mnist_tests)
y_mnist_preds1 = np.argmax(mnist_test_preds1, axis=1)
acc_score_mnist1 = accuracy_score(y_true=modals_test_labels,y_pred=y_mnist_preds1)
print(acc_score_mnist1)

model_svhn_edl1 = resnet18EDL1(class_nums=10,input_shape=(32,32,3))
#model_mnist.summary()
annealing_step = 10*(len(modals_labels)//64)
model_svhn_edl1.compile(optimizer=tf.keras.optimizers.Adam(),loss=CustomMSE(), metrics=['accuracy'])
warm_up_ac = WarmUp(total_steps=annealing_step,global_step_init=0,ac=0,verbose=0)
model_svhn_edl1.fit(modal_svhn_samples, modals_labels_hot, batch_size=64, epochs=10,verbose=1,callbacks=[warm_up_ac])
svhn_test_preds1 = model_svhn_edl1.predict(modal_svhn_tests)
y_svhn_preds1 = np.argmax(svhn_test_preds1, axis=1)
acc_score_svhn1 = accuracy_score(y_true=modals_test_labels,y_pred=y_svhn_preds1)
print(acc_score_svhn1)

In [ ]:
edl1_mnist_ofd_test_preds = model_mnist_edl1.predict(ofd_emnist_tests)
edl1_mnist_ents = st.entropy(edl1_mnist_ofd_test_preds,axis=1)
print(edl1_mnist_ents.mean())
sns.displot(edl1_mnist_ents, kde=True)

In [ ]:
def resnet18EDL1_fuse(class_nums,input_shape1=(28,28,1), input_shape2=(32,32,3)):
    '''主模型'''
    inpt1 = layers.Input(shape=input_shape1)
    inpt2 = layers.Input(shape=input_shape2)
    #layer 1
    x1 = conv2d_bn(inpt1, filters=64, kernel_size=(7,7), strides=2, padding='valid')
    x1 = layers.MaxPool2D(pool_size=(3,3), strides=2)(x1)
    #layer 2
    x1 = basic_bottle(x1, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    x1 = basic_bottle(x1, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    #layer 3
    x1 = basic_bottle(x1, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x1 = basic_bottle(x1, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 4
    x1 = basic_bottle(x1, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x1 = basic_bottle(x1, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 5
    x1 = basic_bottle(x1, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x1 = basic_bottle(x1, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    #GlobalAveragePool
    x1 = layers.GlobalAveragePooling2D()(x1)
    e1 = layers.Dense(class_nums, activation='relu')(x1)
    b1 = layers.Dense(class_nums, activation='softmax')(x1) 

    alpha1 = e1# + 1
    S1 = tf.reduce_sum(alpha1, axis=1, keepdims=True)
    #E1 = tf.reduce_sum(e1, axis=1, keepdims=True)
    #new_e1 = E1*b1
    #alpha1 = new_e1 + 1
    #S1 = tf.reduce_sum(alpha1, axis=1, keepdims=True)
    #new_b1 = new_e1/S1
    #u1 = class_nums/S1

    #layer 1
    x2 = conv2d_bn(inpt2, filters=64, kernel_size=(7,7), strides=2, padding='valid')
    x2 = layers.MaxPool2D(pool_size=(3,3), strides=2)(x2)
    #layer 2
    x2 = basic_bottle(x2, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    x2 = basic_bottle(x2, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    #layer 3
    x2 = basic_bottle(x2, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x2 = basic_bottle(x2, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 4
    x2 = basic_bottle(x2, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x2 = basic_bottle(x2, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 5
    x2 = basic_bottle(x2, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x2 = basic_bottle(x2, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    #GlobalAveragePool
    x2 = layers.GlobalAveragePooling2D()(x2)
    e2 = layers.Dense(class_nums, activation='relu')(x2)
    b2 = layers.Dense(class_nums, activation='softmax')(x2)

    #x2 = layers.Dense(class_nums, activation='linear')(x2)
    #e2 = layers.Activation('relu')(x2)
    #b2 = layers.Activation('softmax')(x2)

    alpha2 = e2# + 1
    S2 = tf.reduce_sum(alpha2, axis=1, keepdims=True)
    #E2 = tf.reduce_sum(e2, axis=1, keepdims=True)
    #new_e2 = E2*b2
    #alpha2 = new_e2 + 1
    #S2 = tf.reduce_sum(alpha2, axis=1, keepdims=True)
    #new_b2 = new_e2/S2
    #u2 = class_nums/S2
    
    S = (S1 + S2) /2
    #b = new_b1*new_b2 + u1*new_b2 + u2*new_b1 
    b = b1*b2
    kappa = tf.reduce_sum(b,axis=1,keepdims=True)
    b_new = b/kappa
    alpha_new = S*b_new+1

    model = tf.keras.Model(inputs=(inpt1,inpt2), outputs=alpha_new)
    return model

In [ ]:
def resnet18EDL2(class_nums,input_shape=(28,28,1)):
    '''主模型'''
    inpt = layers.Input(shape=input_shape)
    #layer 1
    x = conv2d_bn(inpt, filters=64, kernel_size=(7,7), strides=2, padding='valid')
    x = layers.MaxPool2D(pool_size=(3,3), strides=2)(x)
    #layer 2
    x = basic_bottle(x, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    x = basic_bottle(x, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    #layer 3
    x = basic_bottle(x, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 4
    x = basic_bottle(x, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 5
    x = basic_bottle(x, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    #GlobalAveragePool
    x = layers.GlobalAveragePooling2D()(x)
    e = layers.Dense(class_nums, activation='relu')(x)
    alpha = e + 1
 
    model = tf.keras.Model(inputs=inpt, outputs=alpha)
    return model

In [ ]:
model_mnist_edl2 = resnet18EDL2(class_nums=10,input_shape=(28,28,1))
#model_mnist.summary()
annealing_step = 10*(len(modals_labels)//64)
model_mnist_edl2.compile(optimizer=tf.keras.optimizers.Adam(),loss=CustomMSE(), metrics=['accuracy'])
warm_up_ac = WarmUp(total_steps=annealing_step,global_step_init=0,ac=0,verbose=0)
history_edl_mnist2=model_mnist_edl2.fit(modal_mnist_samples, modals_labels_hot, batch_size=64, epochs=10,verbose=1,callbacks=[warm_up_ac])
mnist_test_preds2 = model_mnist_edl2.predict(modal_mnist_tests)
y_mnist_preds2 = np.argmax(mnist_test_preds2, axis=1)
acc_score_mnist2 = accuracy_score(y_true=modals_test_labels,y_pred=y_mnist_preds2)
print(acc_score_mnist2)

In [ ]:
model_svhn_edl2 = resnet18EDL2(class_nums=10,input_shape=(32,32,3))
#model_mnist.summary()
annealing_step = 10*(len(modals_labels)//64)
model_svhn_edl2.compile(optimizer=tf.keras.optimizers.Adam(),loss=CustomMSE(), metrics=['accuracy'])
warm_up_ac = WarmUp(total_steps=annealing_step,global_step_init=0,ac=0,verbose=0)
model_svhn_edl2.fit(modal_svhn_samples, modals_labels_hot, batch_size=64, epochs=10,verbose=1,callbacks=[warm_up_ac])
svhn_test_preds2 = model_svhn_edl2.predict(modal_svhn_tests)
y_svhn_preds2 = np.argmax(svhn_test_preds2, axis=1)
acc_score_svhn2 = accuracy_score(y_true=modals_test_labels,y_pred=y_svhn_preds2)
print(acc_score_svhn2)

In [ ]:
edl2_mnist_ofd_test_preds = model_mnist_edl2.predict(ofd_emnist_tests)
edl2_mnist_ents = st.entropy(edl2_mnist_ofd_test_preds,axis=1)
print(edl2_mnist_ents.mean())
sns.displot(edl2_mnist_ents, kde=True)

In [ ]:
def resnet18EDL2_fuse(class_nums,input_shape1=(28,28,1), input_shape2=(32,32,3)):
    '''主模型'''
    inpt1 = layers.Input(shape=input_shape1)
    inpt2 = layers.Input(shape=input_shape2)
    #layer 1
    x1 = conv2d_bn(inpt1, filters=64, kernel_size=(7,7), strides=2, padding='valid')
    x1 = layers.MaxPool2D(pool_size=(3,3), strides=2)(x1)
    #layer 2
    x1 = basic_bottle(x1, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    x1 = basic_bottle(x1, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    #layer 3
    x1 = basic_bottle(x1, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x1 = basic_bottle(x1, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 4
    x1 = basic_bottle(x1, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x1 = basic_bottle(x1, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 5
    x1 = basic_bottle(x1, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x1 = basic_bottle(x1, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    #GlobalAveragePool
    x1 = layers.GlobalAveragePooling2D()(x1)
    e1 = layers.Dense(class_nums, activation='relu')(x1) 
    alpha1 = e1 + 1
    S1 = tf.reduce_sum(alpha1, axis=1, keepdims=True)
    u1 = class_nums/S1
    b1 = e1/S1

    #layer 1
    x2 = conv2d_bn(inpt2, filters=64, kernel_size=(7,7), strides=2, padding='valid')
    x2 = layers.MaxPool2D(pool_size=(3,3), strides=2)(x2)
    #layer 2
    x2 = basic_bottle(x2, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    x2 = basic_bottle(x2, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    #layer 3
    x2 = basic_bottle(x2, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x2 = basic_bottle(x2, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 4
    x2 = basic_bottle(x2, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x2 = basic_bottle(x2, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 5
    x2 = basic_bottle(x2, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x2 = basic_bottle(x2, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    #GlobalAveragePool
    x2 = layers.GlobalAveragePooling2D()(x2)
    e2 = layers.Dense(class_nums, activation='relu')(x2)
    
    alpha2 = e2 + 1
    S2 = tf.reduce_sum(alpha2, axis=1, keepdims=True)
    u2 = class_nums/S2
    b2 = e2/S2

    #b = new_b1*new_b2 + u1*new_b2 + u2*new_b1 
    b = b1*b2 + b1*u2 + b2*u1
    u = u1*u2
    kappa = tf.reduce_sum(b,axis=1,keepdims=True)+u
    b_new = b/kappa
    u_new = u/kappa
    S = class_nums/u_new
    alpha_new = S*b_new+1
    model = tf.keras.Model(inputs=(inpt1,inpt2), outputs=alpha_new)
    return model

In [ ]:
model_edl2_fuse = resnet18EDL2_fuse(class_nums=10,input_shape1=(28,28,1), input_shape2=(32,32,3))
#model_mnist.summary()
annealing_step = 10*(len(modals_labels)//64)
model_edl2_fuse.compile(optimizer=tf.keras.optimizers.Adam(),loss=CustomMSE(), metrics=['accuracy'])
warm_up_ac = WarmUp(total_steps=annealing_step,global_step_init=0,ac=0,verbose=0)
history_edl2_fuse=model_edl2_fuse.fit((modal_mnist_samples,modal_svhn_samples), modals_labels_hot, batch_size=64, epochs=10,verbose=1,callbacks=[warm_up_ac])
test_preds_fuse2 = model_edl2_fuse.predict((modal_mnist_tests,modal_svhn_tests))
y_preds_fuse2 = np.argmax(test_preds_fuse2, axis=1)
acc_score_fuse2 = accuracy_score(y_true=modals_test_labels,y_pred=y_preds_fuse2)
print(acc_score_fuse2)

In [ ]:
def resnet18EDL12(class_nums,input_shape=(28,28,1)):
    inpt = layers.Input(shape=input_shape)
    #layer 1
    x = conv2d_bn(inpt, filters=64, kernel_size=(7,7), strides=2, padding='valid')
    x = layers.MaxPool2D(pool_size=(3,3), strides=2)(x)
    #layer 2
    x = basic_bottle(x, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    x = basic_bottle(x, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    #layer 3
    x = basic_bottle(x, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 4
    x = basic_bottle(x, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 5
    x = basic_bottle(x, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x = basic_bottle(x, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    #GlobalAveragePool
    x = layers.GlobalAveragePooling2D()(x)
    e = layers.Dense(class_nums, activation='relu')(x)
    b2 = layers.Dense(class_nums, activation='softmax')(x)

    alpha1 = e + 1
    S = tf.reduce_sum(alpha1,axis=1,keepdims=True)
    b1 = e / S
    u1 = class_nums/S
    b = b1*b2+u1*b2

    kappa = tf.reduce_sum(b,axis=1,keepdims=True)
    b_new = (1-u1)*b/kappa
    alpha_new = S*b_new+1
    print(alpha_new) 
    model = tf.keras.Model(inputs=inpt, outputs=alpha_new)
    return model

In [ ]:
model_mnist_edl12 = resnet18EDL12(class_nums=10,input_shape=(28,28,1))
#model_mnist.summary()
annealing_step = 10*(len(modals_labels)//64)
model_mnist_edl12.compile(optimizer=tf.keras.optimizers.Adam(),loss=CustomMSE(), metrics=['accuracy'])
warm_up_ac = WarmUp(total_steps=annealing_step,global_step_init=0,ac=0,verbose=0)
history_edl_mnist12=model_mnist_edl12.fit(modal_mnist_samples, modals_labels_hot, batch_size=64, epochs=10,verbose=1,callbacks=[warm_up_ac])
mnist_test_preds12 = model_mnist_edl12.predict(modal_mnist_tests)
y_mnist_preds12 = np.argmax(mnist_test_preds12, axis=1)
acc_score_mnist12 = accuracy_score(y_true=modals_test_labels,y_pred=y_mnist_preds12)
print(acc_score_mnist12)

In [ ]:
def resnet18EDL12_fuse(class_nums,input_shape1=(28,28,1), input_shape2=(32,32,3)):
    '''主模型'''
    inpt1 = layers.Input(shape=input_shape1)
    inpt2 = layers.Input(shape=input_shape2)
    #layer 1
    x1 = conv2d_bn(inpt1, filters=64, kernel_size=(7,7), strides=2, padding='valid')
    x1 = layers.MaxPool2D(pool_size=(3,3), strides=2)(x1)
    #layer 2
    x1 = basic_bottle(x1, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    x1 = basic_bottle(x1, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    #layer 3
    x1 = basic_bottle(x1, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x1 = basic_bottle(x1, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 4
    x1 = basic_bottle(x1, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x1 = basic_bottle(x1, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 5
    x1 = basic_bottle(x1, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x1 = basic_bottle(x1, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    #GlobalAveragePool
    x1 = layers.GlobalAveragePooling2D()(x1)
    e1 = layers.Dense(class_nums, activation='relu')(x1)
    b12 = layers.Dense(class_nums, activation='softmax')(x1)

    alpha11 = e1 + 1
    S1 = tf.reduce_sum(alpha11,axis=1,keepdims=True)
    b11 = e1 / S1
    u1 = class_nums/S1
    b1 = b11*b12+u1*b12
    kappa1 = tf.reduce_sum(b1,axis=1,keepdims=True)
    b_new1 = (1-u1)*b1/kappa1
    alpha_new1 = S1*b_new1+1

    #layer 1
    x2 = conv2d_bn(inpt2, filters=64, kernel_size=(7,7), strides=2, padding='valid')
    x2 = layers.MaxPool2D(pool_size=(3,3), strides=2)(x2)
    #layer 2
    x2 = basic_bottle(x2, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    x2 = basic_bottle(x2, filters=64, kernel_size=(3,3), strides=1, padding='same', if_baisc=False)
    #layer 3
    x2 = basic_bottle(x2, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x2 = basic_bottle(x2, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 4
    x2 = basic_bottle(x2, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x2 = basic_bottle(x2, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    # layer 5
    x2 = basic_bottle(x2, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    x2 = basic_bottle(x2, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    #GlobalAveragePool
    x2 = layers.GlobalAveragePooling2D()(x2)
    e2 = layers.Dense(class_nums, activation='relu')(x2)
    b22 = layers.Dense(class_nums, activation='softmax')(x2)

    alpha21 = e2 + 1
    S2 = tf.reduce_sum(alpha21,axis=1,keepdims=True)
    b21 = e2 / S2
    u2 = class_nums/S2
    b2 = b21*b22+u2*b22
    kappa2 = tf.reduce_sum(b2,axis=1,keepdims=True)
    b_new2 = (1-u2)*b2/kappa2
    alpha_new2 = S2*b_new2+1

    #b = new_b1*new_b2 + u1*new_b2 + u2*new_b1 
    b = b_new1*b_new2 + b_new1*u2 + b_new2*u1
    u = u1*u2
    kappa = tf.reduce_sum(b,axis=1,keepdims=True)+u
    b_new = b/kappa
    u_new = u/kappa
    S = class_nums/u_new
    alpha_new = S*b_new+1
    model = tf.keras.Model(inputs=(inpt1,inpt2), outputs=alpha_new)
    return model

In [ ]:
model_edl12_fuse = resnet18EDL12_fuse(class_nums=10,input_shape1=(28,28,1), input_shape2=(32,32,3))
#model_mnist.summary()
annealing_step = 10*(len(modals_labels)//64)
model_edl12_fuse.compile(optimizer=tf.keras.optimizers.Adam(),loss=CustomMSE(), metrics=['accuracy'])
warm_up_ac = WarmUp(total_steps=annealing_step,global_step_init=0,ac=0,verbose=0)
history_edl12_fuse=model_edl12_fuse.fit((modal_mnist_samples,modal_svhn_samples), modals_labels_hot, batch_size=64, epochs=10,verbose=1,callbacks=[warm_up_ac])
test_preds_fuse12 = model_edl12_fuse.predict((modal_mnist_tests,modal_svhn_tests))
y_preds_fuse12 = np.argmax(test_preds_fuse12, axis=1)
acc_score_fuse12 = accuracy_score(y_true=modals_test_labels,y_pred=y_preds_fuse12)
print(acc_score_fuse12)

In [ ]:
model_edl12_fuse = resnet18EDL12_fuse(class_nums=10,input_shape1=(28,28,1), input_shape2=(32,32,3))
#model_mnist.summary()
annealing_step = 10*(len(modals_labels)//64)
model_edl12_fuse.compile(optimizer=tf.keras.optimizers.Adam(),loss=CustomMSE(), metrics=['accuracy'])
warm_up_ac = WarmUp(total_steps=annealing_step,global_step_init=0,ac=0,verbose=0)
history_edl12_fuse=model_edl12_fuse.fit((modal_mnist_samples,modal_svhn_samples), modals_labels_hot, batch_size=64, epochs=10,verbose=1,callbacks=[warm_up_ac])
test_preds_fuse12 = model_edl12_fuse.predict((modal_mnist_tests,modal_svhn_tests))
y_preds_fuse12 = np.argmax(test_preds_fuse12, axis=1)
acc_score_fuse12 = accuracy_score(y_true=modals_test_labels,y_pred=y_preds_fuse12)
print(acc_score_fuse12)

In [ ]:
test_preds_fuse12 = model_edl12_fuse.predict((modal_mnist_tests,modal_svhn_tests))
y_preds_fuse12 = np.argmax(test_preds_fuse12, axis=1)
acc_score_fuse12 = accuracy_score(y_true=modals_test_labels,y_pred=y_preds_fuse12)
print(acc_score_fuse12)